# Setup Comprehend Through AWS Console 

<img src="img/comprehend.png" width="80%" align="left">

In [28]:
%store -r

In [54]:
import sys
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
role = sagemaker.get_execution_role()

sm = boto3.Session().client(service_name='sagemaker', region_name=region_name)

In [30]:
comprehend = boto3.client('comprehend')

In [31]:
print(noheader_train_s3_uri)

s3://sagemaker-us-east-2-322537213286/data/amazon_reviews_us_Digital_Software_v1_00_noheader.csv


In [32]:
!aws s3 ls $noheader_train_s3_uri

2020-07-15 07:35:11   13700478 amazon_reviews_us_Digital_Software_v1_00_noheader.csv


# Create Data Access Role for Comprehend

## Create Policy

In [33]:
assume_role_policy_doc = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "comprehend.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
} 

## Create Role and Attach Policies

In [34]:
iam_comprehend_role_name = 'aws_sm_hol_Comprehend'

In [35]:
import json
import boto3
import time

from botocore.exceptions import ClientError

try:
    iam = boto3.client('iam')

    iam_role_comprehend = iam.create_role(
        RoleName=iam_comprehend_role_name,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy_doc),
        Description='SageMAker HOL Comprehend Role'
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        iam_role_comprehend = iam.get_role(RoleName=iam_comprehend_role_name)
        print("Role already exists")
    else:
        print("Unexpected error: %s" % e)
        
time.sleep(30)

Role already exists


In [36]:
comprehend_s3_policy_doc = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "s3:GetObject"
            ],
            "Resource": [
                "arn:aws:s3:::{}/*".format(data_bucket)
            ],
            "Effect": "Allow"
        },
        {
            "Action": [
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(data_bucket)
            ],
            "Effect": "Allow"
        },
        {
            "Action": [
                "s3:PutObject"
            ],
            "Resource": [
                "arn:aws:s3:::{}/*".format(job_bucket)
            ],
            "Effect": "Allow"
        }
    ]
}

print(comprehend_s3_policy_doc)

{'Version': '2012-10-17', 'Statement': [{'Action': ['s3:GetObject'], 'Resource': ['arn:aws:s3:::sagemaker-us-east-2-322537213286/*'], 'Effect': 'Allow'}, {'Action': ['s3:ListBucket'], 'Resource': ['arn:aws:s3:::sagemaker-us-east-2-322537213286'], 'Effect': 'Allow'}, {'Action': ['s3:PutObject'], 'Resource': ['arn:aws:s3:::sagemaker-experiments-us-east-2-322537213286/*'], 'Effect': 'Allow'}]}


# Attach Policy to Role

In [37]:
import time

response = iam.put_role_policy(
    RoleName=iam_comprehend_role_name,
    PolicyName='AWS_SM_HOL_ComprehendPolicyToS3',
    PolicyDocument=json.dumps(comprehend_s3_policy_doc)
)

print(response)

time.sleep(30)

{'ResponseMetadata': {'RequestId': '93226f2a-98e0-4154-8985-e607cac0bff1', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '93226f2a-98e0-4154-8985-e607cac0bff1', 'content-type': 'text/xml', 'content-length': '206', 'date': 'Wed, 15 Jul 2020 09:10:53 GMT'}, 'RetryAttempts': 0}}


# Train the Model

In [38]:
prefix = 'models'

s3_output_job = 's3://{}/{}/{}'.format(job_bucket, prefix, 'comprehend/output')
print(s3_output_job)

s3://sagemaker-experiments-us-east-2-322537213286/models/comprehend/output


In [39]:
iam_role_comprehend_arn = iam_role_comprehend['Role']['Arn']

In [40]:
import datetime
import time

id = str(datetime.datetime.now().strftime("%s"))

training_job = comprehend.create_document_classifier(
    DocumentClassifierName='Amazon-Customer-Reviews-Classifier-'+ id,
    DataAccessRoleArn=iam_role_comprehend_arn,
    InputDataConfig={
        'S3Uri': noheader_train_s3_uri
    },
    OutputDataConfig={
        'S3Uri': s3_output_job
    },
    LanguageCode='en'
)

time.sleep(30)

In [41]:
import time

jobArn = training_job['DocumentClassifierArn']

max_time = time.time() + 3 * 60 * 60 # 3 hours
while time.time() < max_time:
    describe_custom_classifier = comprehend.describe_document_classifier(
        DocumentClassifierArn = jobArn
    )
    status = describe_custom_classifier["DocumentClassifierProperties"]["Status"]
    print("Custom classifier: {}".format(status))
    
    if status == "TRAINED" or status == "IN_ERROR":
        print('')
        print('Status {}'.format(status))
        print('')
        print(describe_custom_classifier["DocumentClassifierProperties"])
        break
        
    time.sleep(10)

Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom cla

<h2><span style="color:red">위 Classifier가 완료되기 전까지 기다려 주시기 바랍니다.</span></h2>

# Show Results of the Classifier

In [42]:
print(describe_custom_classifier["DocumentClassifierProperties"])

{'DocumentClassifierArn': 'arn:aws:comprehend:us-east-2:322537213286:document-classifier/Amazon-Customer-Reviews-Classifier-1594804284', 'LanguageCode': 'en', 'Status': 'TRAINED', 'SubmitTime': datetime.datetime(2020, 7, 15, 9, 11, 24, 610000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2020, 7, 15, 9, 55, 58, 301000, tzinfo=tzlocal()), 'TrainingStartTime': datetime.datetime(2020, 7, 15, 9, 15, 44, 123000, tzinfo=tzlocal()), 'TrainingEndTime': datetime.datetime(2020, 7, 15, 9, 55, 22, 777000, tzinfo=tzlocal()), 'InputDataConfig': {'S3Uri': 's3://sagemaker-us-east-2-322537213286/data/amazon_reviews_us_Digital_Software_v1_00_noheader.csv'}, 'OutputDataConfig': {'S3Uri': 's3://sagemaker-experiments-us-east-2-322537213286/models/comprehend/output/322537213286-CLR-6c79ccfb64d6c02fa039cd892eecde3e/output/output.tar.gz'}, 'ClassifierMetadata': {'NumberOfLabels': 5, 'NumberOfTrainedDocuments': 27905, 'NumberOfTestDocuments': 3100, 'EvaluationMetrics': {'Accuracy': 0.5561, 'Precision': 0.55

In [43]:
model_arn = describe_custom_classifier["DocumentClassifierProperties"]["DocumentClassifierArn"]
print(model_arn)

arn:aws:comprehend:us-east-2:322537213286:document-classifier/Amazon-Customer-Reviews-Classifier-1594804284


In [46]:
import os
#Retrieve the S3URI from the model output and create jobkey variable.
job_output = describe_custom_classifier["DocumentClassifierProperties"]["OutputDataConfig"]["S3Uri"]
print(job_output)

path_prefix = 's3://{}/'.format(job_bucket)

job_key = os.path.relpath(job_output, path_prefix)

print(job_key)

s3://sagemaker-experiments-us-east-2-322537213286/models/comprehend/output/322537213286-CLR-6c79ccfb64d6c02fa039cd892eecde3e/output/output.tar.gz
models/comprehend/output/322537213286-CLR-6c79ccfb64d6c02fa039cd892eecde3e/output/output.tar.gz


In [48]:
#Download the model metrics
s3 = boto3.resource('s3')

s3.Bucket(job_bucket).download_file(job_key, './output.tar.gz')

In [49]:
#Unpack the gzip file
!tar xvzf ./output.tar.gz

output/
output/confusion_matrix.json


In [50]:
import json

with open('./output/confusion_matrix.json') as json_file:
    data = json.load(json_file)
print(json.dumps(data, indent=2, default=str))

{
  "confusion_matrix": [
    [
      412,
      132,
      51,
      13,
      12
    ],
    [
      147,
      273,
      147,
      40,
      13
    ],
    [
      62,
      139,
      298,
      95,
      26
    ],
    [
      12,
      30,
      102,
      305,
      171
    ],
    [
      9,
      9,
      26,
      140,
      436
    ]
  ],
  "labels": [
    "1",
    "2",
    "3",
    "4",
    "5"
  ],
  "type": "multi_class",
  "all_labels": [
    "1",
    "2",
    "3",
    "4",
    "5"
  ]
}


In [55]:
!{sys.executable} -m pip install tabulate

In [56]:
from IPython.display import HTML, display
import tabulate
table = [['', '1', '2', '3', '4', '5', '(Predicted)'],
         ['1', data['confusion_matrix'][0][0], data['confusion_matrix'][0][1], data['confusion_matrix'][0][2], data['confusion_matrix'][0][3], data['confusion_matrix'][0][4]],
         ['2', data['confusion_matrix'][1][0], data['confusion_matrix'][1][1], data['confusion_matrix'][1][2], data['confusion_matrix'][1][3], data['confusion_matrix'][1][4]],
         ['3', data['confusion_matrix'][2][0], data['confusion_matrix'][2][1], data['confusion_matrix'][2][2], data['confusion_matrix'][2][3], data['confusion_matrix'][2][4]],
         ['4', data['confusion_matrix'][3][0], data['confusion_matrix'][3][1], data['confusion_matrix'][3][2], data['confusion_matrix'][3][3], data['confusion_matrix'][3][4]],
         ['5', data['confusion_matrix'][4][0], data['confusion_matrix'][4][1], data['confusion_matrix'][4][2], data['confusion_matrix'][4][3], data['confusion_matrix'][4][4]],
         ['(Actual)']]
display(HTML(tabulate.tabulate(table, tablefmt='html')))

,1,2,3,4,5,(Predicted)
1,412,132,51,13,12,
2,147,273,147,40,13,
3,62,139,298,95,26,
4,12,30,102,305,171,
5,9,9,26,140,436,
(Actual),,,,,,


# Deploy Endpoint

In [57]:
from time import gmtime, strftime, sleep
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())

comprehend_endpoint_name = 'comprehend-inference-ep-' + timestamp_suffix

inference_endpoint_response = comprehend.create_endpoint(
    EndpointName=comprehend_endpoint_name,
    ModelArn = model_arn,
    DesiredInferenceUnits = 1
)

In [58]:
comprehend_endpoint_arn = inference_endpoint_response["EndpointArn"]
print(comprehend_endpoint_arn)

arn:aws:comprehend:us-east-2:322537213286:document-classifier-endpoint/comprehend-inference-ep-15-10-45-14


# Pass Variables to the Next Notebook(s)

In [59]:
%store comprehend_endpoint_arn

Stored 'comprehend_endpoint_arn' (str)


# Predict with Endpoint

In [60]:
describe_response = comprehend.describe_endpoint(
    EndpointArn = comprehend_endpoint_arn
)
print(describe_response)

{'EndpointProperties': {'EndpointArn': 'arn:aws:comprehend:us-east-2:322537213286:document-classifier-endpoint/comprehend-inference-ep-15-10-45-14', 'Status': 'CREATING', 'ModelArn': 'arn:aws:comprehend:us-east-2:322537213286:document-classifier/Amazon-Customer-Reviews-Classifier-1594804284', 'DesiredInferenceUnits': 1, 'CurrentInferenceUnits': 0, 'CreationTime': datetime.datetime(2020, 7, 15, 10, 45, 15, 66000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 7, 15, 10, 45, 15, 66000, tzinfo=tzlocal())}, 'ResponseMetadata': {'RequestId': '11f37d32-1546-448d-9cb8-e7ad6484b5bd', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '11f37d32-1546-448d-9cb8-e7ad6484b5bd', 'content-type': 'application/x-amz-json-1.1', 'content-length': '408', 'date': 'Wed, 15 Jul 2020 10:45:22 GMT'}, 'RetryAttempts': 0}}


In [61]:
import time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_response = comprehend.describe_endpoint(
        EndpointArn = comprehend_endpoint_arn
    )
    status = describe_response["EndpointProperties"]["Status"]
    print("Endpoint: {}".format(status))
    
    if status == "IN_SERVICE" or status == "IN_ERROR":
        break
        
    time.sleep(5)

Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CR

In [62]:
txt = """I loved it!  I will recommend this to everyone."""

response = comprehend.classify_document(
    Text= txt,
    EndpointArn = comprehend_endpoint_arn
)

import json
print(json.dumps(response, indent=2, default=str))

{
  "Classes": [
    {
      "Name": "5",
      "Score": 0.9035000205039978
    },
    {
      "Name": "4",
      "Score": 0.07039999961853027
    },
    {
      "Name": "1",
      "Score": 0.010400000028312206
    }
  ],
  "ResponseMetadata": {
    "RequestId": "a333a279-f02f-4f75-980f-37011d67d654",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "a333a279-f02f-4f75-980f-37011d67d654",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "136",
      "date": "Wed, 15 Jul 2020 10:55:41 GMT"
    },
    "RetryAttempts": 0
  }
}


In [63]:
txt = """It's OK."""

response = comprehend.classify_document(
    Text= txt,
    EndpointArn = comprehend_endpoint_arn
)

import json
print(json.dumps(response, indent=2, default=str))

{
  "Classes": [
    {
      "Name": "3",
      "Score": 0.7032999992370605
    },
    {
      "Name": "4",
      "Score": 0.11869999766349792
    },
    {
      "Name": "2",
      "Score": 0.09589999914169312
    }
  ],
  "ResponseMetadata": {
    "RequestId": "8db42677-8878-4cae-8c53-a140946d19cd",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "8db42677-8878-4cae-8c53-a140946d19cd",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "135",
      "date": "Wed, 15 Jul 2020 10:55:41 GMT"
    },
    "RetryAttempts": 0
  }
}


In [64]:
txt = """Really bad.  I hope they don't make this anymore."""

response = comprehend.classify_document(
    Text= txt,
    EndpointArn = comprehend_endpoint_arn
)

import json
print(json.dumps(response, indent=2, default=str))

{
  "Classes": [
    {
      "Name": "1",
      "Score": 0.38119998574256897
    },
    {
      "Name": "2",
      "Score": 0.3174999952316284
    },
    {
      "Name": "3",
      "Score": 0.22840000689029694
    }
  ],
  "ResponseMetadata": {
    "RequestId": "cb6add5b-a43b-4dc8-ae15-086e7cdd8d39",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "cb6add5b-a43b-4dc8-ae15-086e7cdd8d39",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "135",
      "date": "Wed, 15 Jul 2020 10:55:42 GMT"
    },
    "RetryAttempts": 0
  }
}


# Navigate to Comprehend in the AWS Console
![Comprehend Console](img/comprehend-console.png)